In [1]:
from bs4 import BeautifulSoup
import os
from sklearn.feature_extraction.text import CountVectorizer
from pathlib import Path
import numpy as np
import fnmatch

Now it's time to adapt the code for html we scraped from wikipedia:

In [3]:
# Processing multiple HTML files in a folder
folder_path = r"../Week_2/wikipedia_talk_pages"
documents = []
for filename in os.listdir(folder_path):
    if filename.endswith(".html"):
        file_path = os.path.join(folder_path, filename)
        with open(file_path, "r", encoding="utf-8") as f:
            soup = BeautifulSoup(f.read(), "html.parser")
            result = []

            # page title 
            title = soup.find("span", class_="mw-page-title-main")
            if title:
                result.append(title.get_text(strip=True)) # strip removes leading/trailing whitespace

            # main content
            root = soup.find("div", id="mw-content-text") # we create root to be sure we are in the content section
            content = root.find("div", class_="mw-parser-output") if root else None # it verifies root is not None
            # in content, we look for h2, h3, and p tags that are direct children (not nested deeper)

            if content:
                for el in content.find_all(["h2", "h3", "p"], recursive=False): # recursive=False ensures we only get direct children
                    text = el.get_text(" ", strip=True)
                    if text:
                        result.append(text)

            full_text = "\n\n".join(result) # every wiki page is stored as a single string with double newlines between sections
            if full_text:   # only add non-empty documents    
                documents.append(full_text)

print(f'The number of documents in the list is: {len(documents)}', "Type of documents:", type(documents[0]))
print("First 500 characters of the first document:\n", documents[0][:500])      

The number of documents in the list is: 1225 Type of documents: <class 'str'>
First 500 characters of the first document:
 Hello fellow Wikipedians,

I have just modified one external link on 10th edition of Systema Naturae . Please take a moment to review my edit . If you have any questions, or need the bot to ignore the links, or the page altogether, please visit this simple FaQ for additional information. I made the following changes:

When you have finished reviewing my changes, you may follow the instructions on the template below to fix any issues with the URLs.

This message was posted before February 2018. A


In [4]:
# Operators and/AND, or/OR, not/NOT become &, |, 1 -
# Parentheses are left untouched
# Everything else is interpreted as a term and fed
# through td_matrix[t2i["..."]]
d = {"and": "&", "AND": "&",
         "or": "|", "OR": "|",
         "not": "1 -", "NOT": "1 -",
         "(": "(", ")": ")"}  # operator replacements

In [5]:
def debug_print(*args):
    # If you want debug prints to show up, uncomment the line below: 
    print(*args)
    pass

In [13]:
# k-gram index functions
def build_kgram_index(vocabulary, k=3):
    index = {}
    for term in vocabulary:
        padded = f"${term}$"
        for i in range(len(padded) - k + 1):
            kgram = padded[i:i+k]
            index.setdefault(kgram, set()).add(term)
    return index

# Extract k-grams from wildcard pattern
def kgrams_from_wildcard(pattern, k=3):
    parts = pattern.split("*")
    kgrams = []

    # start
    if parts[0] != "":
        kgrams.append("$" + parts[0][:k-1])

    # end
    if parts[-1] != "":
        kgrams.append(parts[-1][-k+1:] + "$")

    return kgrams

# Expand wildcard pattern using k-gram index
def expand_wildcard(pattern, kgram_index, vocabulary):
    kgrams = kgrams_from_wildcard(pattern)
    if not kgrams:
        return []

    candidates = None
    for kg in kgrams:
        if kg not in kgram_index:
            return []
        if candidates is None:
            candidates = kgram_index[kg].copy()
        else:
            candidates &= kgram_index[kg]

    # post-filtering
    return [t for t in candidates if fnmatch.fnmatch(t, pattern)]

# Expand wildcards in the entire query
def expand_wildcards_in_query(query, kgram_index, vocabulary):
    tokens = query.split()
    new_tokens = []

    for t in tokens:
        if "*" in t or "?" in t:
            expanded = expand_wildcard(t, kgram_index, vocabulary)
            if expanded:
                new_tokens.append("(" + " OR ".join(expanded) + ")")
            else:
                new_tokens.append("( )")  # no match
        else:
            new_tokens.append(t)

    return " ".join(new_tokens)


# rewrite tokens
def rewrite_token(t):
    # If the search term exists in our dictionary of operators, get it, 
    # otherwise find occurrences of the term in `td_matrix``. If the term 
    # is not in our dictionary, then the query results in 0 (since the 
    # term does not occur in any of the documents)
    return d.get(t, f'(td_matrix[t2i["{t}"]] if "{t}" in t2i else empty_row)')



def rewrite_query(query):
    # Rewrite every token in the query
    return " ".join(rewrite_token(t) for t in query.split())

def test_query(query, td_matrix, t2i, documents, kgram_index):
    
    # Generate a row of all zeroes for queries containing words not in our 
    # dictionary
    empty_row = np.matrix(np.repeat(0, td_matrix.shape[1]))

    expanded = expand_wildcards_in_query(query, kgram_index, t2i.keys())
    rewritten = rewrite_query(expanded)

    debug_print("Query: '" + query + "'")
    debug_print("Rewritten:", rewritten)

    # Eval runs the string as a Python command
    # `td_matrix`, `t2i`, and `empty_row` have to be in scope in 
    # order for eval() to work
    eval_result = eval(rewritten)
    debug_print("Matching:", eval_result)

    # Finding the matching document
    hits_matrix = eval_result
    hits_list = list(hits_matrix.nonzero()[1])

    print(f"Found {len(hits_list)} results")

    # Prints the first 500 characters of the matching document
    for i, doc_idx in enumerate(hits_list):
        print(f"Matching doc #{i}: {documents[doc_idx][:500]}")

In [15]:
def main():
    cv = CountVectorizer(lowercase=True, binary=True)

    sparse_matrix = cv.fit_transform(documents)
    # debug_print("Term-document matrix: (?)\n")
    # debug_print(sparse_matrix)

    dense_matrix = sparse_matrix.todense()
    # debug_print("Term-document matrix: (?)\n")
    # debug_print(dense_matrix)

    td_matrix = dense_matrix.T  # .T transposes the matrix
    # debug_print("Term-document matrix:\n")
    # debug_print(td_matrix)

    t2i = cv.vocabulary_
    # debug_print(t2i)

    kgram_index = build_kgram_index(t2i.keys())



    while True:
        query = input("Search for something. If you want to stop your search "
                      "type 'q'. Search: ")
        
        # Remove any leading & trailing whitespace from the query and turn it to 
        # lowercase for case insensitive searching
        query = query.lower().strip()

        # Check for empty queries and continue asking for input if so
        if query == "":
            continue

        if query == "q":
            break
        else:
            print(query)
            # because td_matrix and t2i are defined in main(), also pass these
            # to other functions
            test_query(query, td_matrix, t2i, documents, kgram_index)

def main_using_class():
    import sys
    # This is absolutely a hack and every style guide you will ever find 
    # will tell you to never do this!
    # Unfortunately the "correct" way to do this would be to create a new 
    # GitHub repo, add the package to requirements.txt, manage both repos 
    # simultaniously, etc....
    # ...or just do the hacky way lol it's not like we're getting paid
    sys.path.append("..")
    import Search_Algorithms.boolean
    import typing
    import importlib
    importlib.reload(Search_Algorithms.boolean)

    engine = Search_Algorithms.boolean.BooleanSearchEngine(typing.cast(list[str], documents))

    while True:
        query = input("Search for something. If you want to stop your search "
                      "type 'q'. Search: ")
        if query == "q":
            break

        r = engine.search(query)
        print(f"Results: {len(r)}")
        for i, e in enumerate(r):
            print(f"Result #{i}: {e[:100]}")

main()
# main_using_class()

re*ve
Query: 're*ve'
Rewritten: (td_matrix[t2i["(resistive"]] if "(resistive" in t2i else empty_row) | (td_matrix[t2i["reclusive"]] if "reclusive" in t2i else empty_row) | (td_matrix[t2i["reductive"]] if "reductive" in t2i else empty_row) | (td_matrix[t2i["restorative"]] if "restorative" in t2i else empty_row) | (td_matrix[t2i["reproductive"]] if "reproductive" in t2i else empty_row) | (td_matrix[t2i["refractive"]] if "refractive" in t2i else empty_row) | (td_matrix[t2i["reflective"]] if "reflective" in t2i else empty_row) | (td_matrix[t2i["refimprove"]] if "refimprove" in t2i else empty_row) | (td_matrix[t2i["respective"]] if "respective" in t2i else empty_row) | (td_matrix[t2i["repulsive"]] if "repulsive" in t2i else empty_row) | (td_matrix[t2i["responsive"]] if "responsive" in t2i else empty_row) | (td_matrix[t2i["representative"]] if "representative" in t2i else empty_row) | (td_matrix[t2i["recesive"]] if "recesive" in t2i else empty_row) | (td_matrix[t2i["resolve"]] if "resolve" i